In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split
import time

In [2]:
modelName="/home/alex/projects/1c_ml_regression_diagnostics/cgi-bin/prod_ml_duration_master"
#in_iter_count=
#in_learning_rate=
#in_depth=
in_data_file="/home/alex/projects/1c_ml_regression_diagnostics/cgi-bin/data/master_train_data.csv"
cat_features="field_1,field_2,field_3,field_4,field_5,field_6,field_8,field_11,field_12,field_13"
if len(cat_features)==0:
    cat_features	= None
else:
    cat_features	= cat_features.split(',')
target='field_0'

In [3]:
data_source = pd.read_csv(in_data_file)

In [4]:
fields_count=max(data_source['field'])+1
columns=tuple('field_'+str(i) for i in range(0,int(fields_count)))
data = pd.DataFrame([])
for row in range (0,max(data_source['row'])+1):
    temp=pd.DataFrame(data_source[data_source.row==row]['value']).transpose()
    temp.columns=columns
    data = data.append(temp,ignore_index = True)

data.sort_values(['field_0'], inplace=True)
data.reset_index(drop=True, inplace=True)
data.fillna(0,inplace=True)
data=data.sample(n=len(data))

In [5]:
content=""
X=data.drop('field_0', axis=1)
y=data.field_0
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)			
content+='y: '+str(len(y))+'\n'
content+='y_train: '+str(len(y_train))+'\n'
content+='y_validation: '+str(len(y_validation))+'\n'

#X_test = test_df.drop('field_0', axis=1)
eval_dataset = Pool(data=X_validation,
        label=y_validation,
        cat_features=cat_features)

In [7]:
content = ""
model1 = CatBoostRegressor(
                cat_features=cat_features,
                boost_from_average=True,
                score_function = 'NewtonL2',
                one_hot_max_size = 512,
                depth = 16,
                langevin = True,
                posterior_sampling=True,
                model_shrink_rate = 1/(2*len(y_train)),
                verbose=False
                )

In [ ]:
content = ""
model1 = CatBoostRegressor(
    cat_features=cat_features,
    depth = 16,
    boost_from_average=True,
    score_function = 'NewtonL2',
    one_hot_max_size = 512,
    langevin = True,
    posterior_sampling=True,
    model_shrink_rate = 1/(2*len(y_train)),
    verbose=True    
    #diffusion_temperature = len(y_train)
    )

In [8]:
#model1.fit(X_train, y_train)
model1.fit(X_train, y_train,use_best_model = True,eval_set=eval_dataset)
pred = model1.predict(X_validation)			
params = model1.get_params()
content+=str(params)+'\n'
content+='\n'+params['loss_function']+' loss: '+ str(eval_metric(y_validation.to_numpy(), pred, params['loss_function']))
content+='\nFitted: '+str(model1.is_fitted())
content+='\nModel score:\n'+str(model1.score(X,y))
content

"{'boost_from_average': True, 'posterior_sampling': True, 'langevin': True, 'model_shrink_rate': 0.0019305019305019305, 'score_function': 'NewtonL2', 'cat_features': ['field_1', 'field_2', 'field_3', 'field_4', 'field_5', 'field_6', 'field_8', 'field_11', 'field_12', 'field_13'], 'one_hot_max_size': 512, 'verbose': False, 'loss_function': 'RMSE', 'depth': 16}\n\nRMSE loss: [1587.5398250512176]\nFitted: True\nModel score:\n0.5480719100583147"

In [ ]:
content

In [ ]:
content

In [ ]:
content